<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/finaltask/LatstTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/saiku122/AIJobcolle.git

Cloning into 'AIJobcolle'...
remote: Enumerating objects: 378, done.
remote: Counting objects: 100% (378/378), done.
remote: Compressing objects: 100% (260/260), done.
remote: Total 378 (delta 158), reused 201 (delta 65), pack-reused 0
Receiving objects: 100% (378/378), 11.88 MiB | 14.88 MiB/s, done.
Resolving deltas: 100% (158/158), done.


In [3]:
cd /content/AIJobcolle/MachineLearning/finaltask/

/content/AIJobcolle/MachineLearning/finaltask


★★最終課題★★</br>
〇入力データ</br>
・モデル用：final_hr_analysis_train.csv</br>
・スコア用：final_hr_analysis_test.csv</br>
</br>
〇出力データ</br>
・スコア付データ：</br> 
・モデル：</br>


In [74]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import joblib
from sklearn.metrics import accuracy_score

In [5]:
#インプットファイル
file_model_name = 'final_hr_analysis_train.csv'
file_test_name = 'final_hr_analysis_test.csv'

In [8]:
#  モデル用データ読み込み
df = pd.read_csv('./input/'+ file_model_name,
                 header=0,
                 dtype=object)
ID = df.iloc[:,0] #1列目取得
y=df.iloc[:,1]    #2列目の正解データを取得
X=df.iloc[:,2:]   #3列目以降取得

In [11]:
# one-hot encoding処理
ohe_cols = ['mode_category'] #!!!変更箇所
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_cols)
#display(X_ohe)
X_ohe = X_ohe.dropna(axis=1,how='all')#axis=1　列
#display(X_ohe)
X_ohe_columns = X_ohe.columns.values #モデルデータの列確定

In [13]:
#欠損値の置き換え
imp=SimpleImputer() #平均値で欠損値を補完
imp.fit(X_ohe)
X_ohe = pd.DataFrame(imp.transform(X_ohe),columns=X_ohe_columns) # モデルデータの列を平均値補完適用させ変数を更新

In [15]:
#特徴量抽出
selector = RFECV(estimator=RandomForestClassifier(n_estimators=100,random_state=0), step=0.05) #特徴量をRFE(主成分分析)を用いて次元削減する。(モデルはランダムフォレスト)
selector.fit(X_ohe, y) #RFEした結果を保存
X_ohe_selected = selector.transform(X_ohe) #X_oheに反映
X_ohe_selected = pd.DataFrame(X_ohe_selected, columns=X_ohe_columns[selector.support_]) #削減実行後のデータを再構成

**テストデータの読み込み**<br>
モデル用データのエンコード結果に列名をあわせる

In [94]:
#  テスト用データ読み込み
df_s = pd.read_csv('./input/'+ file_test_name,
                 header=0,
                 dtype=object)
ID_s = df_s.iloc[:,[0]] #1列目取得
y_s=df_s.iloc[:,1]    #2列目の正解データを取得
X_s=df_s.iloc[:,2:]   #3列目以降取得

In [95]:
# one-hot encoding処理
X_ohe_s = pd.get_dummies(X_s, dummy_na=True, columns=ohe_cols)
#display(X_ohe)
X_ohe_s = X_ohe_s.dropna(axis=1,how='all')#axis=1　列
#display(X_ohe)
X_ohe_columns_s = X_ohe_s.columns.values #テストデータの仮確定

In [96]:
#列の整合性チェック
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

diff1 = cols_model - cols_score
print('モデルのみに存在する項目: %s' % diff1)

diff2 = cols_score - cols_model
print('スコアのみに存在する項目: %s' % diff2)

モデルのみに存在する項目: {'mode_category_85131', 'mode_category_85039', 'mode_category_22892', 'mode_category_20727', 'mode_category_22848', 'mode_category_22469', 'mode_category_22030', 'mode_category_84971', 'mode_category_22801', 'mode_category_21626', 'mode_category_21519', 'mode_category_22466', 'mode_category_22962', 'mode_category_22437', 'mode_category_85034', 'mode_category_22326', 'mode_category_35004', 'mode_category_82613', 'mode_category_22967', 'mode_category_84944', 'mode_category_23158', 'mode_category_22664', 'mode_category_85180', 'mode_category_22752', 'mode_category_22656', 'mode_category_84459', 'mode_category_22207', 'mode_category_84050', 'mode_category_21745', 'mode_category_85169', 'mode_category_22037', 'mode_category_22461', 'mode_category_21216', 'mode_category_84507', 'mode_category_22931', 'mode_category_22303', 'mode_category_21843', 'mode_category_21669', 'mode_category_84931', 'mode_category_22261', 'mode_category_79160', 'mode_category_84950', 'mode_category_2264

In [97]:
#列名一致
df_cols_m = pd.DataFrame(None,
                         columns=X_ohe_columns,
                         dtype=float)
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
#display(X_ohe_s2.head(3))

set_Xm = set(X_ohe.columns.values)
set_Xs = set(X_ohe_s.columns.values)

#スコアにのみ存在する項目の削除
X_ohe_s3 = X_ohe_s2.drop(list(set_Xs-set_Xm),axis=1)

#print(X_ohe_s3.shape)
#display(X_ohe_s3.head(3))

#list(set_Xs-set_Xm)

In [98]:
#スコアにのみ存在する項目の削除
X_ohe_s3.loc[:,list(set_Xm-set_Xs)] = X_ohe_s3.loc[:,list(set_Xm-set_Xs)].fillna(0,axis=1)
X_ohe_s3.head(3)

,trips,amount_of_yen,quantity,purchase_period,purchase_recency,purchase_amount_per_trips,regularity,trips_cancellation,trips_cat85099,trips_cat85123,trips_cat22423,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catpost,trips_cat23203,trips_cat22383,mode_category_15044,mode_category_15056,mode_category_20718,mode_category_20725,mode_category_20727,mode_category_20728,mode_category_20971,mode_category_21034,mode_category_21084,mode_category_21116,mode_category_21136,mode_category_21137,mode_category_21181,mode_category_21216,mode_category_21313,mode_category_21314,mode_category_21326,mode_category_21340,mode_category_21417,mode_category_21466,mode_category_21495,mode_category_21519,...,mode_category_85036,mode_category_85039,mode_category_85040,mode_category_85049,mode_category_85059,mode_category_85066,mode_category_85071,mode_category_85099,mode_category_85114,mode_category_85116,mode_category_85118,mode_category_85123,mode_category_85131,mode_category_85132,mode_category_85141,mode_category_85152,mode_category_85159,mode_category_85169,mode_category_85170,mode_category_85175,mode_category_85178,mode_category_85180,mode_category_85183,mode_category_85186,mode_category_85194,mode_category_85202,mode_category_85206,mode_category_85212,mode_category_85230,mode_category_85231,mode_category_85232,mode_category_90125,mode_category_90200,mode_category_90201,mode_category_90214,mode_category_C2,mode_category_D,mode_category_M,mode_category_POST,mode_category_nan
0,5,2080.34,1018,153,2,416.068,30.6,0,1,0,1,1,0,0,0,4,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4,661.64,293,106,42,165.41,26.5,0,0,0,1,0,0,0,0,4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5,1265.48,679,147,28,253.096,29.4,0,0,0,0,0,0,0,0,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
#ソート順の統一
X_ohe_s3 = X_ohe_s3.reindex(X_ohe.columns.values,axis=1)
X_ohe_s3.head(3)

,trips,amount_of_yen,quantity,purchase_period,purchase_recency,purchase_amount_per_trips,regularity,trips_cancellation,trips_cat85099,trips_cat85123,trips_cat22423,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catpost,trips_cat23203,trips_cat22383,mode_category_15044,mode_category_15056,mode_category_20718,mode_category_20725,mode_category_20727,mode_category_20728,mode_category_20971,mode_category_21034,mode_category_21084,mode_category_21116,mode_category_21136,mode_category_21137,mode_category_21181,mode_category_21216,mode_category_21313,mode_category_21314,mode_category_21326,mode_category_21340,mode_category_21417,mode_category_21466,mode_category_21495,mode_category_21519,...,mode_category_85036,mode_category_85039,mode_category_85040,mode_category_85049,mode_category_85059,mode_category_85066,mode_category_85071,mode_category_85099,mode_category_85114,mode_category_85116,mode_category_85118,mode_category_85123,mode_category_85131,mode_category_85132,mode_category_85141,mode_category_85152,mode_category_85159,mode_category_85169,mode_category_85170,mode_category_85175,mode_category_85178,mode_category_85180,mode_category_85183,mode_category_85186,mode_category_85194,mode_category_85202,mode_category_85206,mode_category_85212,mode_category_85230,mode_category_85231,mode_category_85232,mode_category_90125,mode_category_90200,mode_category_90201,mode_category_90214,mode_category_C2,mode_category_D,mode_category_M,mode_category_POST,mode_category_nan
0,5,2080.34,1018,153,2,416.068,30.6,0,1,0,1,1,0,0,0,4,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4,661.64,293,106,42,165.41,26.5,0,0,0,1,0,0,0,0,4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5,1265.48,679,147,28,253.096,29.4,0,0,0,0,0,0,0,0,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
#欠損値処理
X_ohe_s4 = pd.DataFrame(imp.transform(X_ohe_s3),columns=X_ohe_columns)

In [101]:
#RFE適用後の列名のみテストデータから抽出
X_ohe_selected_s=X_ohe_s4.loc[:, X_ohe_columns[selector.support_]]

In [102]:
##テスト実行（最後本当にID_sとくっつけていのか？）
X_ohe_s4

,trips,amount_of_yen,quantity,purchase_period,purchase_recency,purchase_amount_per_trips,regularity,trips_cancellation,trips_cat85099,trips_cat85123,trips_cat22423,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catpost,trips_cat23203,trips_cat22383,mode_category_15044,mode_category_15056,mode_category_20718,mode_category_20725,mode_category_20727,mode_category_20728,mode_category_20971,mode_category_21034,mode_category_21084,mode_category_21116,mode_category_21136,mode_category_21137,mode_category_21181,mode_category_21216,mode_category_21313,mode_category_21314,mode_category_21326,mode_category_21340,mode_category_21417,mode_category_21466,mode_category_21495,mode_category_21519,...,mode_category_85036,mode_category_85039,mode_category_85040,mode_category_85049,mode_category_85059,mode_category_85066,mode_category_85071,mode_category_85099,mode_category_85114,mode_category_85116,mode_category_85118,mode_category_85123,mode_category_85131,mode_category_85132,mode_category_85141,mode_category_85152,mode_category_85159,mode_category_85169,mode_category_85170,mode_category_85175,mode_category_85178,mode_category_85180,mode_category_85183,mode_category_85186,mode_category_85194,mode_category_85202,mode_category_85206,mode_category_85212,mode_category_85230,mode_category_85231,mode_category_85232,mode_category_90125,mode_category_90200,mode_category_90201,mode_category_90214,mode_category_C2,mode_category_D,mode_category_M,mode_category_POST,mode_category_nan
0,5.0,2080.34,1018.0,153.0,2.0,416.068000,30.600000,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4.0,661.64,293.0,106.0,42.0,165.410000,26.500000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5.0,1265.48,679.0,147.0,28.0,253.096000,29.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.0,11056.93,5543.0,103.0,8.0,1579.561429,14.714286,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13.0,63661.49,43090.0,152.0,29.0,4897.037692,11.692308,0.0,4.0,0.0,2.0,2.0,0.0,3.0,4.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,4.0,783.02,501.0,72.0,43.0,195.755000,18.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
803,5.0,1150.47,579.0,92.0,44.0,230.094000,18.400000,0.0,1.0,1.0,1.0,0.0,0.0,1.0

**モデリング**

In [103]:
#アルゴリズム定義
model_name = 'GBC_001'
clf = Pipeline([('scl',StandardScaler()), 
                ('est',GradientBoostingClassifier(random_state=1))
                ])

In [104]:
clf.fit(X_ohe_selected, y.values.ravel())#Xを標準化、勾配ブースティングを適用)

Pipeline(memory=None,
         steps=[('scl',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('est',
                 GradientBoostingClassifier(ccp_alpha=0.0,
                                            criterion='friedman_mse', init=None,
                                            learning_rate=0.1, loss='deviance',
                                            max_depth=3, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100,
                                            n_iter_no_change=None,
                     

In [105]:
results = cross_val_score(clf, X_ohe_selected, y, scoring='roc_auc', cv=5)
print('cv score:', np.average(results), '+-', np.std(results))

cv score: 0.7198053221288514 +- 0.02126526880177877


In [106]:
scores_acc=accuracy_score(y, clf.predict(X_ohe_selected))
scores_acc_s=accuracy_score(y_s, clf.predict(X_ohe_selected_s))
scores_acc
print('モデルデータACC：',scores_acc)
print('評価データACC：',scores_acc_s)

モデルデータACC： 0.9218967921896792
評価データACC： 0.8265179677819083


In [110]:
score = pd.DataFrame(clf.predict_proba(X_ohe_selected_s)[:,0:1], columns=['pred_score'])
ID_s.join(score).to_csv('./output/'+  model_name + '_' + file_test_name + '_with_pred.csv', index=False)

In [113]:
imp = pd.DataFrame([clf.named_steps['est'].feature_importances_], columns=X_ohe_columns[selector.support_])
imp.T.to_csv('./output/'+  model_name + '_feature_importances.csv', index=True)